# 1. Introduction

This notebook contains scripts to run numerical computations and simulations to describe properties of neurons.
It contains cells of text (like this one) and also code-cells (grey background). You can execute
them by selecting and then pressing 
 - SHIFT+ENTER to execute the cell and jump to the next cell
 - or CTRL+ENTER to execute and stay in the cell.
 
Go through the notebook from top to bottom and execute cells by pressing SHIFT-ENTER. 

The parts of the code that you are supposed to change are surrounded by a lines of hashes (######).
Subsequent code cells are folded into little dots and contain code that you do not need to inspect. (The code appears if you click on the dots. It can be collapsed again by clicking the colored bar left of the cell.)

After editing a code-cell, press SHIFT-ENTER twice to set your parameters and then run the computation.

In [ ]:
# This code-cell always need to be excetuted first
# Packages and libraries are loaded for the calculations and plotting
# Brian is a simulator for spiking neural networks. It is written in the Python programming language.
# https://brian2.readthedocs.io

from brian2 import *
from matplotlib import pyplot as plt
%matplotlib ipympl
print('Ok')

The Course contains the following sections:

2. The Goldman equation (or GHK; Goldman–Hodgkin–Katz voltage equation) describes a trans-membrane equilibrium potential. The GHK potential depends on ion specific permeabilities (conductivity).
   - Explore the GHK equation and how it relates to the individual Nernst potentials.
   - Inspect the development of conductances in the HH model and estimate the GHK potential.
3. The Cable model
   - Compute a constant current injection in an axon.
   - Explore the parameters and corresponding length constant of the model.
4. The HH model
   - Calculate the AP velocity for different parameters.
   - Does the velocity relate to the length constant $\lambda$ ?
5. HH to investigate ion channels

   
# 2. Goldman equation

The transmembrane potential of a semipermeable membrane is dependent on the concentration [X] and conductivity $g_X$ of ions.
When more than one ion species is considered around a semipermeable membrane, the equilibrium potential is given by the Goldmann equation. Considering only Sodium and Potassium, the Goldmann eq. is:

$$
V_{m, GHK} ={\frac  {RT}{F}}\ln {\left({\frac  {g_{{{\text{Na}}}}[{\text{Na}}^{{+}}]_{{\mathrm  {out}}}+g_{{{\text{K}}}}[{\text{K}}^{{+}}]_{{\mathrm  {out}}}}{g_{{{\text{Na}}}}[{\text{Na}}^{{+}}]_{{\mathrm  {in}}}+g_{{{\text{K}}}}[{\text{K}}^{{+}}]_{{{\mathrm  {in}}}}}}\right)}
$$
$$
%V_m ={\frac  {RT}{F}}\ln {\left({\frac  {P_{{{\text{Na}}}}[{\text{Na}}^{{+}}]_{{\mathrm  {out}}}+P_{{{\text{K}}}}[{\text{K}}^{{+}}]_{{\mathrm  {out}}}+P_{{{\text{Cl}}}}[{\text{Cl}}^{{-}}]_{{\mathrm  {in}}}}{P_{{{\text{Na}}}}[{\text{Na}}^{{+}}]_{{\mathrm  {in}}}+P_{{{\text{K}}}}[{\text{K}}^{{+}}]_{{{\mathrm  {in}}}}+P_{{{\text{Cl}}}}[{\text{Cl}}^{{-}}]_{{\mathrm  {out}}}}}\right)}
$$

## 2.1 Steady state conditions across the membrane
First, we implement the GHK equation to explore the resulting membrane potential. To this aim, we manipulate the ion's conductivities. When conductivities change, the membrane strives towards a new membrane potential.

We define commen standard values for the conductivities and concentrations.


In [ ]:
# Common conductance values of a nerve fiber in the resting state
g_Na_standard = 0.05
g_K_standard = 1.0

# Common values for ion concentrations (Arbitrary Units, the units cancel out in the GHK equation)
Na_in_standard = 15
Na_out_standard = 145
K_in_standard = 140
K_out_standard = 4

In [ ]:
############################################## TODO ############################################## 
# The following values are plotted in the cell below.
# YOUR TASK: Add more values to see how Vm changes.
g_na_values = [0.05, 0.5]
g_k_values = [1.0, 10]

# Here you can overwrite the standard values for the concentrations
# When no values are given, the standard values are used
Ion_Concentrations = [
    {'label': 'Standard'
    },
    #{'label': 'Decreased K_in',
    # 'K_in': 1,
    #},
]
##################################################################################################

In [ ]:
def vm_goldmann(g_Na=g_Na_standard,
                g_K=g_K_standard,
                Na_in=Na_in_standard,
                Na_out=Na_out_standard,
                K_in=K_in_standard,
                K_out=K_out_standard):
    """The Goldman-Hodgkin-Katz equation.
       input: Conductivities and concentrations in arbitrary units.
       returns: Vm in volt."""

    # RT/F at room temperature
    prefactor = 26.7e-3  # Volt

    # the conductivity weighted concentration ratio sum
    gc_ratio = (g_Na*Na_out + g_K * K_out)/(g_Na*Na_in + g_K * K_in)

    # Return the transmembrane potential
    vm = prefactor * np.log(gc_ratio)
    return vm

fig, axs = plt.subplots(1, 2, figsize=(9, 5))
g_Na_list = np.array(g_na_values)
g_K_list = np.array(g_k_values) #np.logspace(np.log10(1), 3)

for Ion_Concentration in Ion_Concentrations:
    label = Ion_Concentration.pop('label')
    axs[0].plot(g_Na_list, 1e3*vm_goldmann(g_Na=g_Na_list, **Ion_Concentration), 'o', label=label)
    axs[1].plot(g_K_list, 1e3*vm_goldmann(g_K=g_K_list, **Ion_Concentration), 'o', label=label)

axs[0].semilogx()
axs[0].set_ylabel("$V_m (mV)$")
axs[0].set_title("$V_m$ with increasing $g_{Na}$")
axs[0].set_xlabel("$g_{Na}$")
axs[0].set_ylim(-100, 100)
axs[0].grid()
axs[0].legend()

axs[1].semilogx()
axs[1].set_ylabel("$V_m (mV)$")
axs[1].set_title("$V_m$ with increasing $g_{K}$")
axs[1].set_xlabel("$g_{K}$")
axs[1].set_ylim(-100, 100)
axs[1].grid()
axs[1].legend()

fig.tight_layout()

## 2.2 The full range of the Goldman equation

In [ ]:
g_Na_list = np.logspace(-3, 3)
g_ratio_list = g_Na_list/g_K_standard
fig, ax = plt.subplots(1,1,figsize=(9,5))
ax.plot(g_ratio_list, 1e3*vm_goldmann(g_Na=g_ratio_list))

ax.semilogx()
ax.set_ylabel("$V_m (mV)$")
ax.set_title("$V_m$ as a continous function of $g_{Na} / g_K$")
ax.grid()
ax.set_xlabel("$g_{Na}/g_K$");
plt.show()

## 2.3 Time course of conductivities in the HH model

In [ ]:
from scipy import stats

start_scope()
defaultclock.dt = 0.02*ms

morpho = Cylinder(length=10*cm, diameter=2*238*um, n=100)

El = 10.613*mV

ENa = 115*mV
EK = -12*mV

gl = 0.4*msiemens/cm**2
gNa0 = 120*msiemens/cm**2
gK = 36*msiemens/cm**2

# HH equations
eqs = '''
Im = gl * (El-v) + gNa * m**3 * h * (ENa-v) + gK * n**4 * (EK-v) : amp/meter**2
I : amp (point current) # applied current
dm/dt = alpham * (1-m) - betam * m : 1
dn/dt = alphan * (1-n) - betan * n : 1
dh/dt = alphah * (1-h) - betah * h : 1
alpham = (0.1/mV) * 10.0*mV/exprel((-v+25.0*mV)/(10.0*mV))/ms : Hz
betam = 4 * exp(-v/(18.0*mV))/ms : Hz
alphah = 0.07 * exp(-v/(20.0*mV))/ms : Hz
betah = 1/(exp((-v+30.0*mV) / (10.0*mV)) + 1)/ms : Hz
alphan = (0.01/mV) * 10.0*mV/exprel((-v+10.0*mV)/(10.0*mV))/ms : Hz
betan = 0.125*exp(-v/(80.0*mV))/ms : Hz
gNa : siemens/meter**2
gNaTotal = gNa * m**3 *h : siemens/meter**2
gKTotal = gK * n**4 : siemens/meter**2
'''

neuron = SpatialNeuron(morphology=morpho, model=eqs, method="exponential_euler",
                       #refractory="m > 0.4", threshold="m > 0.5",
                       Cm=1*uF/cm**2, Ri=35.4*ohm*cm)
neuron.v = 0*mV
neuron.h = 1
neuron.m = 0
neuron.n = .5
neuron.I = 0*amp
neuron.gNa = gNa0

M = StateMonitor(neuron, ('v', 'gNaTotal', 'gKTotal'), record=[50])

la = neuron.space_constant[0]
print('------------------- Running simulation ---------------------')
print("Length constante: %s" % la.in_best_unit(2))
print("Time constante:", (neuron.Cm[0]/gl).in_best_unit(2))
print('------------------------  Start ----------------------------')

run(20*ms, report='text')
neuron.I[0] = 1*uA # current injection at one end
run(3*ms)
neuron.I = 0*amp
run(30*ms, report='text')


fig, ax = plt.subplots(1,1,figsize=(9,5))
i = 0#50
gNatot = M.gNaTotal.T[:, i]
ax.plot(M.t/ms, M.gNaTotal.T[:, i]/(siemens/meter**2), label = '$g_{Na}$')
ax.plot(M.t/ms, M.gKTotal.T[:, i]/(siemens/meter**2), label = '$g_{K}$')
ax.set_ylabel('conductivity [siemens/m$^2$]')
ax.set_xlabel('time')
ax.legend()

# 3. Current injection at passive Cable

What happens when current is injected in a thin nerve fiber, how does the current spread?

The answer is given by the core-conductor- or cable-model. 
It is a very general model that can be dated back to Lord Kelvin who derived it to describe transatlatic communication cables. Bioelectricity, e.g. in nerve fibers, obeys the same physical laws.

In this model the ion species are not distinguished, there is just one current (of course still with a capacitive and a resistive component). We use a library called Brian to numerically solve the equations.

The following code inject current at one end of a finite (not infinite) passive fiber. 
You may change capacity, Radius, leak conductivity etc.

- Run the next code-cell to create a new figure. (SHIFT+ENTER)
- Run the subsequent code-cell multiple times with different parameters and labels. (preferably use CTRL+ENTER)
  (Run the first code-cell again) by selecting (click on it) and then pressing SHIFT+ENTER or CTRL+ENTER

In [ ]:
# This code-cell creates a new empty figure
# Re-run to clear figure
fig, ax = plt.subplots(1,1, figsize=(9, 5))

In [ ]:
############################################## TODO ##############################################
# Define the list of parameters that are used
# The following parameter pairs are used in the simulation below.
# Add more value pairs to explore the cable model.
# Format:    [Cm        , gL]
label = 'standard'
Cm = 1*uF/cm**2
gL = 1e-4*siemens/cm**2
diameter = 1*um
##################################################################################################

# Morphology
length = 2*mm
Ri = 150*ohm*cm
N = 200

# Passive channels
EL = -70*mV
eqs = '''
Im = gL * (EL - v) : amp/meter**2
I : amp (point current)
'''

start_scope()
defaultclock.dt = 0.1*ms
morpho = Cylinder(diameter=diameter, length=length, n=N)

neuron = SpatialNeuron(morphology=morpho, model=eqs, Cm=Cm, Ri=Ri,
                       method='exponential_euler')
neuron.v = EL
M = StateMonitor(neuron, 'v', record=[0, 50])


print("%s:" % label)
print("Cm =", Cm.in_best_unit(3))
print("gL =", gL)

la = neuron.space_constant[0]
print("Length constant = %s" % la.in_best_unit(2))
print("Length constant (theory) = %s" % (sqrt(1/(gL*pi*diameter*Ri/(pi*(diameter/2)**2))).in_best_unit(2)))
print("Time constant = %s" % (Cm/gL).in_best_unit(2))

# Currrent injection at one end
neuron.I[0] = 0.02*nA

# Running an plotting
report = None
run(.5*ms, report=report)
run(4.5*ms, report=report)
run(45*ms, report=report)

ln = ax.plot(M.t/ms, M.v.T[:, 0]/mV, '-',
             label=label+' '+str(neuron.morphology.distance[M.record[0]].in_best_unit(2)))
ax.plot(M.t/ms, M.v.T[:, 1]/mV, '--', color=ln[0].get_color(),
        label=label+' '+str(neuron.morphology.distance[M.record[1]].in_best_unit(2)))


ax.legend(title="Position along the axon x (mm)",
          loc="best", ncol=2)  # , label=x[M.record])
ax.set_xlabel('Time since beginning of current injection (ms)')
ax.set_ylabel('Membrane potential $V_m(x, t)$ (mV)')
fig.tight_layout()

# 4. HH Action Potential

An AP is a rapid and propagating jump in the transmembrane potential. There are many models to describe such a process.
The most famous one is the HH model, which is a very detailed purely electrical model.
In the first Section it was found that a change in ion specific permeabilities effects the membrane potential. 

The HH model uses the Goldman eq. and describes the permeabilities with 
voltage and time dependent expressions.
It has more than 20 constant parameters and 3 internal variables (m, n, h).
With so many parameters, various AP shapes can be reproduced.
However, these properties can not be monitored or controlled so easily, and it is not reaally clear what m, n and h represent in an actual neuron.


The following code simuates a HH AP along a long axon, and calculates the propagation velocity.

- Run the following code by selecting (click on it) and then pressing SHIFT+ENTER or CTRL+ENTER
- whats the AP velocity?

The nerve is almost half a millimeter thick and still the AP speed is rather disenchanting (should be around 12 m/s)
You have heard of myelin which wraps around axons and increases the AP propagation speed.
We can include the insulating effect of myelin by decreasing the leak condictivity $g_L$.
(Note, divide $g_L$ by 2 would double the so called Electrotonic length or "length parameter")

- Try to make the AP double its speed by decresing $g_L$.
  What is the relation between length parameter and propagation speed?

- Change $C_m$ to reach the desired velocity.

- Bonus for fun: Are there other ways to accelerate the AP?

In [ ]:
# The following parameter sets are used.
HH_parameters = [
    {
    'Cm': 1 * uF / cm**2,
    'Ri': 35.4 * ohm * cm,
    'gl': 0.4 * msiemens / cm**2,
    'diameter': 2 * 238 * um,
    'label': 'Standard'
    }, 
############################################## TODO ##############################################
# Add more parameter sets to investigate the effect on the propagation velocity.
    {
    'Cm': 1 * uF / cm**2,
    'Ri': 35.4 * ohm * cm,
    'gl': 0.00004 * msiemens / cm**2,
    'diameter': 2 * 238 * um,
    'label': 'no leak'
    },
##################################################################################################
]

In [ ]:
from scipy import stats

fig, ax = plt.subplots(1,1, figsize=(9,5))

for HH_parameter in HH_parameters:
    
    Cm = HH_parameter['Cm']
    Ri = HH_parameter['Ri']
    gl = HH_parameter['gl']
    diameter = HH_parameter['diameter']
    label = HH_parameter['label']
    
    start_scope()
    defaultclock.dt = 0.05*ms

    morpho = Cylinder(length=10*cm, diameter=diameter, n=500)

    El = 10.613*mV
    ENa = 115*mV
    EK = -12*mV
    gNa0 = 120*msiemens/cm**2
    gK = 36*msiemens/cm**2
    gl = gl

    # HH equations
    eqs = '''
    Im = gl * (El-v) + gNa * m**3 * h * (ENa-v) + gK * n**4 * (EK-v) : amp/meter**2
    I : amp (point current) # applied current
    dm/dt = alpham * (1-m) - betam * m : 1
    dn/dt = alphan * (1-n) - betan * n : 1
    dh/dt = alphah * (1-h) - betah * h : 1
    alpham = (0.1/mV) * 10.0*mV/exprel((-v+25.0*mV)/(10.0*mV))/ms : Hz
    betam = 4 * exp(-v/(18.0*mV))/ms : Hz
    alphah = 0.07 * exp(-v/(20.0*mV))/ms : Hz
    betah = 1/(exp((-v+30.0*mV) / (10.0*mV)) + 1)/ms : Hz
    alphan = (0.01/mV) * 10.0*mV/exprel((-v+10.0*mV)/(10.0*mV))/ms : Hz
    betan = 0.125*exp(-v/(80.0*mV))/ms : Hz
    gNa : siemens/meter**2
    '''

    neuron = SpatialNeuron(morphology=morpho, model=eqs, method="exponential_euler",
                           refractory="m > 0.4", threshold="m > 0.5",
                           Cm=Cm, Ri=Ri)
    neuron.v = 0*mV
    neuron.h = 1
    neuron.m = 0
    neuron.n = .5
    neuron.I = 0*amp
    neuron.gNa = gNa0
    M = StateMonitor(neuron, ('v', 'gNa'), record=True)
    spikes = SpikeMonitor(neuron)


    report=None
    print('------------------- Running simulation ---------------------')
    #la = neuron.space_constant[0]
    #print("Init: Length constante: %s" % la.in_best_unit(2))
    #print("Init: Time constante:", (neuron.Cm[0]/gl).in_best_unit(2))
    run(20*ms, report=report)
    la = neuron.space_constant[0]
    print("Relaxed: Length constante: %s" % la.in_best_unit(2))
    print("Relaxed: Time constante:", (neuron.Cm[0]/gl).in_best_unit(2))
    neuron.I[0] = 3*uA # current injection at one end
    #la = neuron.space_constant[0]
    #print("Post stimulation: Length constante: %s" % la.in_best_unit(2))
    #print("Post stimulation: Time constante:", (neuron.Cm[0]/gl).in_best_unit(2))
    run(3*ms, report=report)
    neuron.I = 0*amp
    run(30*ms, report=report)

    # Calculation of velocity
    slope, intercept, r_value, p_value, std_err = stats.linregress(spikes.t/second,
                                                    neuron.distance[spikes.i]/meter)
    print('------------------------  Done  ----------------------------')
    print("  Velocity = %.2f m/s" % slope)
    
    i = morpho.n//3
    ln = ax.plot(M.t/ms, M.v[i,:]/mV, 
         label=label+' '+str(morpho.distance[i]-morpho.distance[0]))
    i = 2*morpho.n//3
    ax.plot(M.t/ms, M.v[i,:]/mV, '--', color=ln[0].get_color(),
         label=label+' '+str(morpho.distance[i]-morpho.distance[0]))
    legend(title='Position along the axon')
    ylabel('Membrane potential (mV)')
    xlabel('Time (ms)')

# 5. HH to investigate ion channels

John von Neuman once said:

    Give me three parameters and I can Fit an Elephant, give me four and i can make him wiggle his trunk.

Short describtion of the following cells
 - HH original
 - HH One parameter is changed
 - plot both AP
 
 Run these three code-cells and look at the output

In [ ]:
# HH code with standard parameters

HH_parameter = {
    'Cm': 1 * uF / cm**2,
    'Ri': 35.4 * ohm * cm,
    'gl': 0.4 * msiemens / cm**2,
    'diameter': 2 * 238 * um,
    'label': 'Standard'
}

Cm = HH_parameter['Cm']
Ri = HH_parameter['Ri']
gl = HH_parameter['gl']
diameter = HH_parameter['diameter']
label = HH_parameter['label']

start_scope()
defaultclock.dt = 0.05 * ms

morpho = Cylinder(length=10 * cm, diameter=diameter, n=500)

El = 10.613 * mV
ENa = 115 * mV
EK = -12 * mV
gNa0 = 120 * msiemens / cm**2
gK = 36 * msiemens / cm**2
gl = gl

# HH equations
eqs = '''
Im = gl * (El-v) + gNa * m**3 * h * (ENa-v) + gK * n**4 * (EK-v) : amp/meter**2
I : amp (point current) # applied current
dm/dt = alpham * (1-m) - betam * m : 1
dn/dt = alphan * (1-n) - betan * n : 1
dh/dt = alphah * (1-h) - betah * h : 1
alpham = (0.1/mV) * 10.0*mV/exprel((-v+25.0*mV)/(10.0*mV))/ms : Hz
betam = 4 * exp(-v/(18.0*mV))/ms : Hz
alphah = 0.07 * exp(-v/(20.0*mV))/ms : Hz
betah = 1/(exp((-v+30.0*mV) / (10.0*mV)) + 1)/ms : Hz
alphan = (0.01/mV) * 10.0*mV/exprel((-v+10.0*mV)/(10.0*mV))/ms : Hz
betan = 0.125*exp(-v/(80.0*mV))/ms : Hz
gNa : siemens/meter**2
'''

neuron = SpatialNeuron(morphology=morpho,
                       model=eqs,
                       method="exponential_euler",
                       refractory="m > 0.4",
                       threshold="m > 0.5",
                       Cm=Cm,
                       Ri=Ri)
neuron.v = 0 * mV
neuron.h = 1
neuron.m = 0
neuron.n = .5
neuron.I = 0 * amp
neuron.gNa = gNa0
M = StateMonitor(neuron, ('v', 'gNa'), record=True)
spikes = SpikeMonitor(neuron)

la = neuron.space_constant[0]
print('------------------- Running simulation ---------------------')
print("Length constante: %s" % la.in_best_unit(2))
print("Time constante:", (neuron.Cm[0] / gl).in_best_unit(2))
print('------------------------  Start ----------------------------')

run(20 * ms, report='text')
neuron.I[0] = 3 * uA  # current injection at one end
run(3 * ms)
la = neuron.space_constant[0]
print("Length constante: %s" % la.in_best_unit(2))
neuron.I = 0 * amp
run(30 * ms, report='text')

# Calculation of velocity
slope, intercept, r_value, p_value, std_err = stats.linregress(
    spikes.t / second, neuron.distance[spikes.i] / meter)
print('------------------------  Done  ----------------------------')
print("  Velocity = %.2f m/s" % slope)


In [ ]:
# HH code with modified parameters

HH_parameter = {
    'Cm': 1 * uF / cm**2,
    'Ri': 35.4 * ohm * cm,
    'gl': 0.4 * msiemens / cm**2,
    'diameter': 2 * 238 * um,
    'label': 'Standard'
}

Cm = HH_parameter['Cm']
Ri = HH_parameter['Ri']
gl = HH_parameter['gl']
diameter = HH_parameter['diameter']
label = HH_parameter['label']

start_scope()
defaultclock.dt = 0.05 * ms

morpho = Cylinder(length=10 * cm, diameter=diameter, n=500)

El = 10.613 * mV
ENa = 115 * mV
EK = -12 * mV
gNa0 = 120 * msiemens / cm**2
gK = 9 * msiemens / cm**2
gl = gl

# HH equations
eqs = '''
Im = gl * (El-v) + gNa * m**3 * h * (ENa-v) + gK * n**4 * (EK-v) : amp/meter**2
I : amp (point current) # applied current
dm/dt = alpham * (1-m) - betam * m : 1
dn/dt = alphan * (1-n) - betan * n : 1
dh/dt = alphah * (1-h) - betah * h : 1
alpham = (0.1/mV) * 10.0*mV/exprel((-v+25.0*mV)/(10.0*mV))/ms : Hz
betam = 4 * exp(-v/(18.0*mV))/ms : Hz
alphah = 0.07 * exp(-v/(20.0*mV))/ms : Hz
betah = 1/(exp((-v+30.0*mV) / (10.0*mV)) + 1)/ms : Hz
alphan = (0.01/mV) * 10.0*mV/exprel((-v+10.0*mV)/(10.0*mV))/ms : Hz
betan = 0.125*exp(-v/(80.0*mV))/ms : Hz
gNa : siemens/meter**2
'''

neuron = SpatialNeuron(morphology=morpho,
                       model=eqs,
                       method="exponential_euler",
                       refractory="m > 0.4",
                       threshold="m > 0.5",
                       Cm=Cm,
                       Ri=Ri)
neuron.v = 0 * mV
neuron.h = 1
neuron.m = 0
neuron.n = .5
neuron.I = 0 * amp
neuron.gNa = gNa0
M_drugged = StateMonitor(neuron, ('v', 'gNa'), record=True)
spikes = SpikeMonitor(neuron)

la = neuron.space_constant[0]
print('------------------- Running simulation ---------------------')
print("Length constante: %s" % la.in_best_unit(2))
print("Time constante:", (neuron.Cm[0] / gl).in_best_unit(2))
print('------------------------  Start ----------------------------')

run(20 * ms, report='text')
neuron.I[0] = 3 * uA  # current injection at one end
run(3 * ms)
la = neuron.space_constant[0]
print("Length constante: %s" % la.in_best_unit(2))
neuron.I = 0 * amp
run(30 * ms, report='text')

# Calculation of velocity
slope, intercept, r_value, p_value, std_err = stats.linregress(
    spikes.t / second, neuron.distance[spikes.i] / meter)
print('------------------------  Done  ----------------------------')
print("  Velocity = %.2f m/s" % slope)

In [ ]:
fig, ax = plt.subplots(1,1)

# Normal
i = morpho.n//3
imax = np.argmax(M.v[i,:])
ln = ax.plot((M.t-M.t[imax])/ms, M.v[i,:]/mV, 
     label='Normal')
legend(title='Position along the axon')
ylabel('Membrane potential (mV)')
xlabel('Time (ms)')


# Drug applied
i = morpho.n//3

imax = np.argmax(M_drugged.v[i,:])
ln = ax.plot((M_drugged.t-M_drugged.t[imax])/ms, M_drugged.v[i,:]/mV, 
     label='Drug applied')
legend()
ylabel('Membrane potential (mV)')
xlabel('Time (ms)')

ax.set_xlim(-5, 10)

# 6. Supplementary material: 

## Analytic expression for the velocity
The effective $g^\star$ is around a third of the peak value of $g_{Na}$

In [ ]:
Cm = 1 * uF/cm**2
diameter = 238*2*um
Ri = 36*ohm*cm
gstar = 100*siemens/meter**2 

v_p = (1/(Cm)) * sqrt(diameter * gstar/(8*Ri))
v_p.in_best_unit(1)

## The conductivities and corresponding Goldman equilibrium

The following section is just a demonstration how the above provided plots are generated. Understanding the following sections is not part of the Couse.

In [ ]:
from scipy import stats

start_scope()
defaultclock.dt = 0.02*ms

morpho = Cylinder(length=10*cm, diameter=2*238*um, n=100)

El = 10.613*mV

ENa = 115*mV
EK = -12*mV

gl = 0.4*msiemens/cm**2
gNa0 = 120*msiemens/cm**2
gK = 36*msiemens/cm**2

# HH equations
eqs = '''
Im = gl * (El-v) + gNa * m**3 * h * (ENa-v) + gK * n**4 * (EK-v) : amp/meter**2
I : amp (point current) # applied current
dm/dt = alpham * (1-m) - betam * m : 1
dn/dt = alphan * (1-n) - betan * n : 1
dh/dt = alphah * (1-h) - betah * h : 1
alpham = (0.1/mV) * 10.0*mV/exprel((-v+25.0*mV)/(10.0*mV))/ms : Hz
betam = 4 * exp(-v/(18.0*mV))/ms : Hz
alphah = 0.07 * exp(-v/(20.0*mV))/ms : Hz
betah = 1/(exp((-v+30.0*mV) / (10.0*mV)) + 1)/ms : Hz
alphan = (0.01/mV) * 10.0*mV/exprel((-v+10.0*mV)/(10.0*mV))/ms : Hz
betan = 0.125*exp(-v/(80.0*mV))/ms : Hz
gNa : siemens/meter**2
gNaTotal = gNa * m**3 *h : siemens/meter**2
gKTotal = gK * n**4 : siemens/meter**2
'''

neuron = SpatialNeuron(morphology=morpho, model=eqs, method="exponential_euler",
                       #refractory="m > 0.4", threshold="m > 0.5",
                       Cm=1*uF/cm**2, Ri=35.4*ohm*cm)
neuron.v = 0*mV
neuron.h = 1
neuron.m = 0
neuron.n = .5
neuron.I = 0*amp
neuron.gNa = gNa0

M = StateMonitor(neuron, ('v', 'gNaTotal', 'gKTotal'), record=[50])

la = neuron.space_constant[0]
print('------------------- Running simulation ---------------------')
print("Length constante: %s" % la.in_best_unit(2))
print("Time constante:", (neuron.Cm[0]/gl).in_best_unit(2))
print('------------------------  Start ----------------------------')

run(20*ms, report='text')
neuron.I[0] = 1*uA # current injection at one end
run(3*ms)
neuron.I = 0*amp
run(30*ms, report='text')


fig, ax = plt.subplots(1,1,figsize=(9,5))
i = 0#50
gNatot = M.gNaTotal.T[:, i]
ax.plot(M.t/ms, M.gNaTotal.T[:, i]/(siemens/meter**2), label = '$g_{Na}$')
ax.plot(M.t/ms, M.gKTotal.T[:, i]/(siemens/meter**2), label = '$g_{K}$')
ax.set_ylabel('conductivity [siemens/m$^2$]')
ax.set_xlabel('time')
ax.legend()

In [ ]:
prefactor = 26.7e-3 * volt
K_out_HH = K_in_standard *exp(-12*mV/prefactor )
#vm_goldmann(g_K=1e99, K_out=K_out_HH)*volt
Na_out_HH = Na_in_standard *exp(115*mV/prefactor )
#vm_goldmann(g_Na=1e99, Na_out=Na_out_HH)*volt

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(9, 5), sharex=True)
i = 0#50
gNatot = M.gNaTotal.T[:, i]
axs[0].plot(M.t/ms, M.gNaTotal.T[:, i]/(siemens/meter**2), label = '$g_{Na}$')
axs[0].plot(M.t/ms, M.gKTotal.T[:, i]/(siemens/meter**2), label = '$g_{K}$')

axs[0].set_ylabel('conductivity [siemens/m$^2$]')
axs[0].set_xlabel('time')
axs[0].legend()

axs[1].plot(M.t/ms, M.v.T[:, i]/mV, label='Membrane potential')
axs[1].plot(M.t/ms, vm_goldmann(g_Na=M.gNaTotal.T[:, i],
                                g_K=M.gKTotal.T[:, i],
                                Na_out=Na_out_HH,
                                K_out=K_out_HH,
                                )/mV, label = 'Goldman equilibrium (leak-free)')

axs[1].set_ylabel('Potential [mV]')
axs[1].set_xlabel('time')
axs[1].legend()
fig.tight_layout()